In [1]:
import json
import pandas as pd
import numpy as np
import time
import requests
import logging
from collections import defaultdict

In [2]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("economic-test")
logger.setLevel(logging.INFO)

In [3]:
def get_information(method, params):
    url = 'https://api.s0.os.hmny.io/'
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    r = requests.post(url, headers=headers, data = json.dumps(data))
    content = json.loads(r.content)
    return content

In [4]:
def getBlockNumber():
    method = "hmy_blockNumber"
    params = []
    num = get_information(method, params)['result']
    return int(num, 16)

In [5]:
def getLastBlockOfCurrentEpoch():
    method = 'hmy_getStakingNetworkInfo'
    params = []
    return get_information(method, params)['result']['epoch-last-block']

In [6]:
def getCurrentAndLastBlock():
    block = getBlockNumber()
    last_block = getLastBlockOfCurrentEpoch()
    print("current and last block numbers", block, last_block)
    return block, last_block

In [7]:
def getAllValidatorInformation():
    method = 'hmy_getAllValidatorInformation'
    params = [-1]
    return get_information(method, params)['result'] 

In [8]:
def getSuperCommittees():
    method = "hmy_getSuperCommittees"
    params = []
    return get_information(method, params)['result']['current']

In [9]:
def diffAndFilter(map1, map2):
    map3 = dict()
    for k, v in map2.items():
        if k in map1:
            map3[k] = v - map1[k]
    return map3

In [10]:
def diffAndFilter2(map1, map2):
    map3 = dict()
    for key, val in map2.items():
        diff = dict()
        for k, v in map2[key].items():
            if v - map1[key][k] != 0:
                diff[k] = v - map1[key][k]
        map3[key] = diff
    return map3

In [11]:
def R5_test(single):
    logger.info(f"Test-R5: Reward given out to block signers sums up to the total block reward")
    logger.info(f"check sum over earned-reward diff of all keys per shard = 28")
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the first block in the new epoch")
        while block < last_block+1:
            block = getBlockNumber()
    logger.info(f"current block {block}, will begin collecting infos...")

    earn_rewards_prev = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                shard = key['shard-id']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_prev[shard][bls_key] = earn_reward
                
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    earn_rewards_curr = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                shard = key['shard-id']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_curr[shard][bls_key] = earn_reward
    earn_reward_diff = diffAndFilter2(earn_rewards_prev, earn_rewards_curr) 
    for shard, diff in earn_reward_diff.items():
        reward = sum(diff.values())
        if format(reward, '.20e') != format(block_reward, '.20e'):
            logger.warning(f"Test-R5: Fail")
            logger.warning(f"shard: {shard}, block: {block}, sum of earned reward: {reward:.20e}, block reward: {block_reward:.20e}\n")
            flag = False
    
    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [12]:
R5_test(True)

INFO:economic-test:Test-R5: Reward given out to block signers sums up to the total block reward
INFO:economic-test:check sum over earned-reward diff of all keys per shard = 28
INFO:economic-test:current and last block numbers: 5840, 5849
INFO:economic-test:current block 5840, will begin collecting infos...


current and last block numbers 5840 5849


INFO:economic-test:new block 5841 reached, will begin testing...





(False, None)

In [13]:
def R5_test(single):
    logger.info(f"check sum over earned-reward diff of all keys per shard = validator reward diff")
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the first block in the new epoch")
        while block < last_block+1:
            block = getBlockNumber()
    logger.info(f"current block {block}, will begin collecting infos...")
    acc_reward_prev = defaultdict(int)
    earn_rewards_prev = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['metrics']:
            addr = i['validator']['address']
            acc_reward_prev[addr] = i['lifetime']['reward-accumulated']
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_prev[addr][bls_key] = earn_reward
                acc_reward_prev[addr] = i['lifetime']['reward-accumulated']

    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    earn_rewards_curr = defaultdict(dict)
    acc_reward_curr = defaultdict(int)
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    for i in validator_infos:
        if i['metrics']:
            addr = i['validator']['address']
            acc_reward_curr[addr] = i['lifetime']['reward-accumulated']
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_curr[addr][bls_key] = earn_reward

    earn_reward_diff = diffAndFilter2(earn_rewards_prev, earn_rewards_curr) 
    val_reward_diff = diffAndFilter(acc_reward_prev, acc_reward_curr)
    for addr, diff in earn_reward_diff.items():
        reward = sum(diff.values())
        val_reward = val_reward_diff[addr]
        if format(reward, '.20e') != format(val_reward, '.20e'):
            logger.warning(f"Test-R5: Fail")
            logger.warning(f"validator: {addr}, sum of earned reward: {reward:.20e}, validator reward: {val_reward:.20e}\n")
            flag = False

    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [14]:
R5_test(True)

INFO:economic-test:check sum over earned-reward diff of all keys per shard = validator reward diff
INFO:economic-test:current and last block numbers: 5841, 5849
INFO:economic-test:current block 5841, will begin collecting infos...


current and last block numbers 5841 5849


INFO:economic-test:new block 5842 reached, will begin testing...
INFO:economic-test:Test-R5: Succeed



(True, None)

In [15]:
def R5_test(single):
    logger.info(f"check sum over earned-reward diff of all keys per shard = 28")
    block, last_block = getCurrentAndLastBlock()
    logger.info(f"current and last block numbers: {block}, {last_block}")
    if block == last_block or block == last_block -1:
        logger.info(f"current at the last block or last second block, wait until the first block in the new epoch")
        while block < last_block+1:
            block = getBlockNumber()
    logger.info(f"current block {block}, will begin collecting infos...")

    earn_rewards_prev = defaultdict(int)
    validator_infos = getAllValidatorInformation()
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_prev[bls_key] = earn_reward
                
    next_block = block + 1
    while block < next_block:
        block = getBlockNumber()
    logger.info(f"new block {block} reached, will begin testing...")
    flag = True
    # get the validator info and compute validator rewards
    earn_rewards_curr = defaultdict(dict)
    voting_power = defaultdict(dict)
    validator_infos = getAllValidatorInformation()
    block_reward = 28e18
    for i in validator_infos:
        if i['metrics']:
            for k in i['metrics']['by-bls-key']:
                key = k['key']
                bls_key = key['bls-public-key']
                earn_reward = k['earned-reward']
                earn_rewards_curr[bls_key] = earn_reward
                perc = float(key['group-percent'])
                voting_power[bls_key] = perc
    earn_reward_diff = diffAndFilter(earn_rewards_prev, earn_rewards_curr) 
    for k, v in earn_reward_diff.items():
        reward = voting_power[k] * block_reward
        if format(v, '.20e') != format(reward, '.20e'):
            logger.warning(f"Test-R5: Fail")
            logger.warning(f"bls-key: {k}, earned reward: {v:.20e}, voting-power * 28: {reward:.20e}\n")
            flag = False
  
    if single:
        curr_test = None
    else:
        curr_test = R6_test
    if flag:
        logger.info(f"Test-R5: Succeed\n")
        return True, curr_test
    else:
        return False, curr_test

In [16]:
R5_test(True)

INFO:economic-test:check sum over earned-reward diff of all keys per shard = 28
INFO:economic-test:current and last block numbers: 5842, 5849
INFO:economic-test:current block 5842, will begin collecting infos...


current and last block numbers 5842 5849


INFO:economic-test:new block 5843 reached, will begin testing...


































(False, None)

In [17]:
res = getSuperCommittees()['quorum-deciders']
for shard, value in res.items():
    member = value['committee-members']
    for i in member:
        if i['earning-account'] == 'one1237cvlujddq9f68s94344rptsuwx0h7ek5lcnc':
            print(f"voting-power-%: {i['voting-power-%']}, voting-power-unnormalized: {i['voting-power-unnormalized']}, effective-stake: {i['effective-stake']}")
            

voting-power-%: 0.011037544815846501, voting-power-unnormalized: 0.034492327549520317, effective-stake: 688575150000000000000000.000000000000000000


In [18]:
res = getAllValidatorInformation()[2]['metrics']['by-bls-key'][0]['key']
print(f"group-percent: {res['group-percent']}, overall-percent: {res['overall-percent']}, effective-stake: {res['effective-stake']}")

group-percent: 0.034492327549520317, overall-percent: 0.011037544815846501, effective-stake: 688575150000000000000000.000000000000000000
